In [19]:
import pandas as pd
import numpy as np


In [21]:
# 데이터 불러오기
df = pd.read_csv("/Users/nelllio/Desktop/Machine_Learning/playground-series/train.csv")
print(df.shape)
df.head()
#df 변수 지정 후 head()를 통해 5행까지 데이터 확인

(750000, 9)


,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [5]:
train = df.info()
print(train)
# count에 non-null로 결측치 없는 것 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 51.5+ MB
None


In [23]:
# ===============================
# 0. 라이브러리 (동일)
# ===============================
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import numpy as np, pandas as pd

# ===============================
# 1. 데이터 & FE (동일)
# ===============================

def create_golden_features(df):
    df = df.copy()
    df['Duration_x_HeartRate']   = df['Duration'] * df['Heart_Rate']
    df['Age_adjusted_HeartRate'] = df['Heart_Rate'] / (df['Age'] + 1)
    df['BMI']                    = df['Weight'] / (df['Height'] / 100) ** 2
    df['Metabolic_Intensity']    = df['Heart_Rate'] * df['Body_Temp']
    df['Steps_per_min']          = df['Heart_Rate'] / (df['Duration'] / 60)
    df['Weight_to_Age']          = df['Weight'] / (df['Age'] + 1)
    df['HeartRate_to_BMI']       = df['Heart_Rate'] / (df['BMI'] + 1)
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    return df

df = create_golden_features(df)
features = [
    'Duration','Heart_Rate','BMI','Body_Temp',
    'Duration_x_HeartRate','Metabolic_Intensity',
    'Age','Height','Weight',
    'Steps_per_min','Age_adjusted_HeartRate',
    'Weight_to_Age','HeartRate_to_BMI','Sex'
]
X, y = df[features], df['Calories']

# ===============================
# 2. RMSLE 스코어러 + 5-fold CV
# ===============================
def rmsle(y_true, y_pred):
    y_pred = np.maximum(y_pred, 0.1)                      # log 안정화
    return np.sqrt(mean_squared_error(np.log1p(y_true),
                                      np.log1p(y_pred)))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)  # 음수 반환(작을수록 좋음)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ===============================
# 3. Base learner – 미세 튜닝
#    (n_estimators ↑, learning_rate ↓)
# ===============================
xgb = XGBRegressor(
        n_estimators=500, learning_rate=0.05,
        max_depth=6, subsample=0.9, colsample_bytree=0.9,
        n_jobs=-1, random_state=42, verbosity=0)

lgb = LGBMRegressor(
        n_estimators=500, learning_rate=0.05,
        max_depth=-1, subsample=0.9, colsample_bytree=0.9,
        n_jobs=-1, random_state=42)

cat = CatBoostRegressor(
        n_estimators=500, learning_rate=0.05,
        depth=8, l2_leaf_reg=3,
        verbose=0, random_state=42)

# ===============================
# 4. 스태킹 정의 (동일)
# ===============================
stack = StackingRegressor(
    estimators=[('xgb', xgb), ('lgb', lgb), ('cat', cat)],
    final_estimator=RidgeCV(),
    n_jobs=-1
)

# ===============================
# 5-1. 교차검증 RMSLE 확인
# ===============================
cv_scores = cross_val_score(stack, X, np.log1p(y),   # log1p 변환 ⇐ 동일 기준
                            cv=kf, scoring=rmsle_scorer,
                            n_jobs=-1)
print(f"📊 5-fold CV RMSLE: {(-cv_scores.mean()):.4f} ± {cv_scores.std():.4f}")

# ===============================
# 5-2. 홀드아웃 학습/평가
# ===============================
X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42)
print("\n📚 모델 학습 중...")
stack.fit(X_train, np.log1p(y_train))
y_pred = np.expm1(stack.predict(X_val))
print(f"🎯 Hold-out RMSLE: {rmsle(y_val, y_pred):.4f}")

# ===============================
# 6. Kaggle 제출용 코드 추가
# ===============================
print("\n🚀 Kaggle 제출 파일 생성 중...")

# 6-1. 테스트 데이터 로드
test_df = pd.read_csv("/Users/nelllio/Desktop/Machine_Learning/playground-series/test.csv")

# 6-2. 테스트 데이터에 동일한 피처 엔지니어링 적용
test_df = create_golden_features(test_df)
X_test = test_df[features]

# 6-3. 전체 훈련 데이터로 최종 모델 학습
print("📚 전체 훈련 데이터로 최종 모델 학습 중...")
stack.fit(X, np.log1p(y))

# 6-4. 테스트 데이터 예측
print("🔮 테스트 데이터 예측 중...")
test_predictions = np.expm1(stack.predict(X_test))

# 6-5. submission 파일 생성
submission = pd.DataFrame({
    'id': test_df['id'],
    'Calories': test_predictions
})

# 6-6. submission 파일 저장
submission_path = "/Users/nelllio/Desktop/Machine_Learning/playground-series/submission.csv"
submission.to_csv(submission_path, index=False)

print(f"✅ Submission 파일이 생성되었습니다: {submission_path}")
print(f"📊 예측된 칼로리 범위: {test_predictions.min():.2f} ~ {test_predictions.max():.2f}")
print(f"📊 평균 예측 칼로리: {test_predictions.mean():.2f}")
print(f"📋 Submission 형태:")
print(submission.head())

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2143
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 14
[LightGBM] [Info] Start training from score 4.140724
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2143
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 14
[LightGBM] [Info] Start training from score 4.141163
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.052867 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total 